In [11]:
# coding: utf-8
import pandas as pd
from pathlib import Path
root = Path('.')
root = root.absolute()
test_cases_root = root / "testcases"
results_root = root / "fmcad_results_2023-05-06_10-28-32/"

In [33]:
print("root: ", root)
print("test_cases_root: ", test_cases_root)
print("results_root: ", results_root)
# cwd
print("cwd: ", Path.cwd())

root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether
test_cases_root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether/testcases
results_root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether/fmcad_results_2023-05-06_10-28-32
cwd:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether


In [19]:
test_cases = [y for x in test_cases_root.iterdir() for y in x.iterdir() if y.suffix == ".th"]
rel_cases = [x.relative_to(test_cases_root) for x in test_cases]
configs = [x.name for x in results_root.iterdir()]
jsons = {n: pd.read_csv(results_root /  n/ "stats.csv") for n in configs}
# Rename columns such that Unnamed: 0 is the test suite nameL
jsons = {n: j.rename(columns={"Unnamed: 0": "test_suite"}) for n, j in jsons.items()}
# Assert test suite column of first configurations contains at least one case of clam
assert any(jsons[configs[0]]["test_suite"].str.contains("clam"))

In [36]:
# Start adding missing files by first adding an error column
for j in jsons.values():
    j["error"] = None

# Now add the missing files. For each missing case find the corresponding error file
for n, j in jsons.items():
    # Find missing cases by relative path to rel_cases
    in_df = j['relative_path'].unique()
    missing_cases = [r for r in rel_cases if r not in in_df]
    for fn in missing_cases:
        errors = (results_root / n / fn.with_suffix('.err')).read_text()
        j.loc[len(j)] = {"relative_path": fn, "error": errors}
        

In [44]:
print(jsons[configs[0]]['error'].value_counts())

error
Timeout Exception                                                                                                                                                                                                                                                                                                                      71
16:30:35 [ERROR] Unrecognized token `(` found at 2707:2708\nExpected one of "then"\nthread 'main' panicked at 'Please implement error handling', src/thesy/semantics.rs:123:17\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n                                                           18
thread 'main' panicked at 'called `Result::unwrap()` on an `Err` value: "Found a list in the head position: [String(\"x\"), String(\"Nat\")]"', src/bin/expl_experiment.rs:125:28\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n                                                                     13
16:30:35 [ERROR] 